# 7-1. Python MySQL 연동
python 데이터베이스 연동 패키지인 pymysql을 활용하여 데이터베이스와 연동하는 방법

## 준비사항
- MySQL8 설치
- root 계정 비밀번호는 123qwe!@#으로 가정함
- myschool 데이터베이스 생성
- 샘플 테이블 존배 가정

# #01. 패키지 설치

In [1]:
pip install --upgrade pymysql

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# #02. 연동준비
## 1) 패키지 참조

In [2]:
import pymysql
from pandas import DataFrame

## 2) 데이터베이스 연동에 필요한 접속 정보

In [3]:
HOSTNAME = 'localhost'
PORT = 3306
USERNAME = 'root'
PASSWORD = '123qwe!@#'
DATABASE = 'myschool'
CHARSET = 'utf8'

# #03. 데이터베이스 접속하기

In [5]:
dbcon = pymysql.connect(host=HOSTNAME, port=PORT, user=USERNAME, password=PASSWORD,
                       db=DATABASE, charset=CHARSET)
print('데이터베이스 접속 성공')
print(type(dbcon))
print(dbcon)

데이터베이스 접속 성공
<class 'pymysql.connections.Connection'>


# #04. SQL 수행하기
## 1) 커서객체 생성
커서란 SQL을 실행하는 기능을 갖는 객체이다. 

`cursor()` 함수에 파라미터를 생략할 경우 SQL에 전달되는 파라미터를 리스트타입으로 관리해야 한다. (인덱스 번호에 따른 위치 지정)<br>
`pymysql.cursors.DictCursor`를 지정할 경우 SQL에 전달되는 파라미터를 딕셔너리 형태로 관리할 수 있다.

In [6]:
cursor = dbcon.cursor(pymysql.cursors.DictCursor)

## 2) 데이터 저장하기
`INSERT` 문은 SQL 수행 후 AUTO_INCREMENT로 지정된 PK 값을 반환받을 수 있다.

In [7]:
# 저장할 데이터 설정
dname = 'Python'
loc = '1004호'

# 저장을 위한 SQL 실행 (문자열 데이터도 홑따옴표 사용 안함)
sql = "INSERT INTO department (dname, loc) VALUES (%s, %s)"
cursor.execute(sql, (dname, loc))

# insert 시에 생성된 PK 얻기
# -> 반드시 commit 전에 조회해야 한다. (UPDATE/DELETE 불필요)
new_deptno = dbcon.insert_id()
print("생성된 학과번호 : %d" % new_deptno)

# 변경사항 저장하기
dbcon.commit()

생성된 학과번호 : 310


## 3) 데이터 수정하기
앞에서 저장한 데이터의 학과명과 위치 수정하기
> ※ 주의사항 : SQL에 저장되는 모든 변수는 `%s` 를 사용해야 하고, 데이터타입은 `string`이어야 한다.

In [10]:
# 수정할 데이터 설정
dname = '빅데이터'
loc = '1004호'

# 수정을 위한 SQL 실행 (문자열 데이터도 홑따옴표 사용 안함)
sql = "UPDATE department SET dname=%s, loc=%s WHERE deptno=%s"
edit_count = cursor.execute(sql, (dname, loc, str(new_deptno)))

print("수정된 데이터 수 : %d" % edit_count)

# 변경사항 저장하기
dbcon.commit()

수정된 데이터 수 : 1


## 4) 데이터 삭제하기

In [11]:
# 삭제를 위한 SQL 실행 (문자열 데이터도 홑따옴표 사용 안함)
sql = "DELETE FROM department WHERE deptno=%s"
delete_count = cursor.execute(sql, (str(new_deptno)))

print("삭제된 데이터 수 : %d" % delete_count)

# 변경사항 저장하기
dbcon.commit()

삭제된 데이터 수 : 1


## 5) 단일행 데이터 조회하기
딕셔너리 키 값 = SQL 문에서 별칭을 적용할 경우 별칭으로 설정, 별칭을 사용하지 않으면 원래 컬럼이름<br>
SQL 문 안에서 줄바꿈이 필요한 경우 전체 구문을 삼중 따옴표로 감싸야 한다.

In [12]:
# 단일행 조회를 위한 SQL구문 형식 설정
sql = """SELECT deptno AS `학과번호`, dname AS `학과이름`, loc AS `학과위치`
        FROM department WHERE deptno=%s"""

# %d에 202라는 값을 적용하여  SQL 실행
# -> sql구문에 치환문자가 없을 경우 두 번째 파라미터 생략 가능
cursor.execute(sql, (202))

# 조회 결과 한 줄을 딕셔너리로 변환
result = cursor.fetchone()
result

{'학과번호': 202, '학과이름': '기계공학과', '학과위치': '4호관'}

## 6) 다중 데이터 조회하기
### SQL문 실행하기
`fetchall()` 함수를 사용하면 동일한 구조의 딕셔너리를 원소로 갖는 리스트로 반환된다.

In [13]:
# 다중행 조회를 위한 SQL구문 형식 설정
sql = "SELECT deptno, dname, loc FROM department"
cursor.execute(sql)

# 조회 결과 전체를 딕셔너리를 원소로 갖는 리스트로 변환
result = cursor.fetchall()
result

[{'deptno': 101, 'dname': '컴퓨터공학과', 'loc': '1호관'},
 {'deptno': 102, 'dname': '멀티미디어학과', 'loc': '2호관'},
 {'deptno': 201, 'dname': '전자공학과', 'loc': '3호관'},
 {'deptno': 202, 'dname': '기계공학과', 'loc': '4호관'},
 {'deptno': 204, 'dname': '수정학과', 'loc': '6호관'}]

### 데이터 조회 결과를 데이터 프레임으로 변환

In [14]:
df = DataFrame(result)
df.set_index('deptno', inplace=True)
df

,dname,loc
deptno,,
101,컴퓨터공학과,1호관
102,멀티미디어학과,2호관
201,전자공학과,3호관
202,기계공학과,4호관
204,수정학과,6호관


# #05. 데이터베이스 접속 해제

In [15]:
dbcon.close()
print('데이터베이스 접속 해제')

데이터베이스 접속 해제
